In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg._interface import MatrixLinearOperator, _CustomLinearOperator
from scipy.sparse.linalg import splu

import jlinops

In [2]:
class SubsamplingOperator(_CustomLinearOperator):
    """Represents a subsampling operator.
    """

    def __init__(self, mask):
        """
        mask: an array representing a binary mask for subsampling, where 1 means observed and 0 means unobserved.
        """

        # Bind mask
        self.mask = mask
        
        # Figure out input and output shapes
        input_shape = self.mask.size
        num_nz = self.mask.sum()
        output_shape = num_nz

        # Other
        self.idxs_observed = np.where(self.mask == 1)

        # Define matvec and rmatvec
        def _matvec(x):
            return x[self.idxs_observed]
        
        def _rmatvec(x):
            tmp = np.zeros(input_shape)
            tmp[self.idxs_observed] = x
            return tmp

        super().__init__( (output_shape, input_shape), _matvec, _rmatvec)


In [12]:
z = np.arange(10)

In [13]:
mask = np.zeros_like(z)
mask[3:6] = 1

In [14]:
op = SubsamplingOperator(mask)

In [17]:
op.T @ (op @ z)

array([0., 0., 0., 3., 4., 5., 0., 0., 0., 0.])

In [19]:
op

<3x10 SubsamplingOperator with dtype=float64>

In [21]:
Z = np.random.normal(size=(10,8))

In [22]:
op @ Z

array([[ 2.37761391,  1.06071361,  1.72052361, -0.38591186, -0.00944256,
        -0.48328705,  0.09914962, -0.14124938],
       [ 0.60353896, -1.42941432,  0.49668285,  0.92159309, -0.35457249,
         1.52475159, -0.04940085, -0.94798063],
       [ 0.42308792, -0.44220155, -0.57637946, -1.76496579, -0.37384834,
        -0.60020262, -0.44054927,  0.25691646]])